# Analysis using the data collected in investment_analysis.ipynp

# Imports

In [2]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import preprocessing

%matplotlib inline

# Reading in CSV Files

### Reading in and correctly indexing the Crime DataFrame

In [4]:
# Reading in crime + Population csv
crime_path = Path("../Data Exploration/resources/crime_out.csv")
crime_data = pd.read_csv(crime_path, index_col="city_description", infer_datetime_format=True, parse_dates=True)

# Removing irrelevant columns
crime_data = crime_data.drop(columns= ["actual_murder", "actual_index_violent"])
crime_data.head()

,date,cbsa_code,population,actual_all_crimes
city_description,,,,
"Austin-Round Rock, TX Metro Area",2014-01-01,12420,1941049,6686.0
"Charlotte-Concord-Gastonia, NC-SC Metro Area",2014-01-01,16740,2373749,8141.0
"Columbia, SC Metro Area",2014-01-01,17900,804684,3172.0
"Dallas-Fort Worth-Arlington, TX Metro Area",2014-01-01,19100,6945276,23204.0
"Las Vegas-Henderson-Paradise, NV Metro Area",2014-01-01,29820,2066423,8875.0


In [5]:
# To get only month_year and city_description as index
crime_data['month_year'] = pd.to_datetime(crime_data['date']).dt.to_period('M')
crime_data = crime_data.reset_index()
crime_data = crime_data.set_index(['month_year', 'city_description'])
crime_data.head()

date  \
month_year city_description                                           
2014-01    Austin-Round Rock, TX Metro Area              2014-01-01   
           Charlotte-Concord-Gastonia, NC-SC Metro Area  2014-01-01   
           Columbia, SC Metro Area                       2014-01-01   
           Dallas-Fort Worth-Arlington, TX Metro Area    2014-01-01   
           Las Vegas-Henderson-Paradise, NV Metro Area   2014-01-01   

                                                         cbsa_code  \
month_year city_description                                          
2014-01    Austin-Round Rock, TX Metro Area                  12420   
           Charlotte-Concord-Gastonia, NC-SC Metro Area      16740   
           Columbia, SC Metro Area                           17900   
           Dallas-Fort Worth-Arlington, TX Metro Area        19100   
           Las Vegas-Henderson-Paradise, NV Metro Area       29820   

                                                         population  \
month_year city_description                                           
2014-01    Austin-Round Rock, TX Metro Area                 1941049   
           Charlotte-Concord-Gastonia, NC-SC Metro Area     2373749   
           Columbia, SC Metro Area                           804684   
           Dallas-Fort Worth-Arlington, TX Metro Area       6945276   
           Las Vegas-Henderson-Paradise, NV Metro Area      2066423   

                                                         actual_all_crimes  
month_year city_description                                                 
2014-01    Austin-Round Rock, TX Metro Area                         6686.0  
           Charlotte-Concord-Gastonia, NC-SC Metro Area             8141.0  
           Columbia, SC Metro Area                                  3172.0  
           Dallas-Fort Worth-Arlington, TX Metro Area              23204.0  
           Las Vegas-Henderson-Paradise, NV Metro Area              8875.0

### Reading in and correctly indexing the Cumulative Retunes DataFrame

In [8]:
# Reading in Cumulative Returns csv
returns_path = Path("../Data Exploration/resources/returns_by_city_out.csv")
returns_data = pd.read_csv(returns_path, index_col="date", infer_datetime_format=True, parse_dates=True)

# Formatting column names
returns_data = returns_data.rename(columns={
    "Unnamed: 1": "city_description",
    "0": "cum_returns"
})

returns_data = returns_data.reset_index()
returns_data = returns_data.set_index(['city_description'])

returns_data.head()

,date,cum_returns
city_description,,
"Phoenix-Mesa-Scottsdale, AZ Metro Area",2014-01-31,0.009568
"Riverside-San Bernardino-Ontario, CA Metro Area",2014-01-31,0.014861
"Orlando-Kissimmee-Sanford, FL Metro Area",2014-01-31,0.009348
"Dallas-Fort Worth-Arlington, TX Metro Area",2014-01-31,0.005632
"Austin-Round Rock, TX Metro Area",2014-01-31,0.006739


In [9]:
# To get only month_year and city_description as index
returns_data['month_year'] = pd.to_datetime(returns_data['date']).dt.to_period('M')
returns_data = returns_data.reset_index()
returns_data = returns_data.set_index(['month_year', 'city_description'])
returns_data.head()

date  \
month_year city_description                                             
2014-01    Phoenix-Mesa-Scottsdale, AZ Metro Area          2014-01-31   
           Riverside-San Bernardino-Ontario, CA Metro Area 2014-01-31   
           Orlando-Kissimmee-Sanford, FL Metro Area        2014-01-31   
           Dallas-Fort Worth-Arlington, TX Metro Area      2014-01-31   
           Austin-Round Rock, TX Metro Area                2014-01-31   

                                                            cum_returns  
month_year city_description                                              
2014-01    Phoenix-Mesa-Scottsdale, AZ Metro Area              0.009568  
           Riverside-San Bernardino-Ontario, CA Metro Area     0.014861  
           Orlando-Kissimmee-Sanford, FL Metro Area            0.009348  
           Dallas-Fort Worth-Arlington, TX Metro Area          0.005632  
           Austin-Round Rock, TX Metro Area                    0.006739

# Joining DataFrames

In [10]:
combined_df = pd.concat([crime_data,returns_data], axis='columns', join='inner')
combined_df

date  \
month_year city_description                                                 
2014-05    Riverside-San Bernardino-Ontario, CA Metro Area     2014-05-01   
           Columbia, SC Metro Area                             2014-05-01   
2014-01    Phoenix-Mesa-Scottsdale, AZ Metro Area              2014-01-01   
2018-08    Orlando-Kissimmee-Sanford, FL Metro Area            2018-08-01   
2017-10    Las Vegas-Henderson-Paradise, NV Metro Area         2017-10-01   
2017-08    Charlotte-Concord-Gastonia, NC-SC Metro Area        2017-08-01   
2016-08    Las Vegas-Henderson-Paradise, NV Metro Area         2016-08-01   
2017-06    Nashville-Davidson--Murfreesboro--Franklin, TN ...  2017-06-01   
2017-11    Riverside-San Bernardino-Ontario, CA Metro Area     2017-11-01   
2017-12    Dallas-Fort Worth-Arlington, TX Metro Area          2017-12-01   
2015-07    Charlotte-Concord-Gastonia, NC-SC Metro Area        2015-07-01   
2016-09    Riverside-San Bernardino-Ontario, CA Metro Area     2016-09-01   
2017-05    Dallas-Fort Worth-Arlington, TX Metro Area          2017-05-01   
2017-04    Charlotte-Concord-Gastonia, NC-SC Metro Area        2017-04-01   
2018-08    Columbia, SC Metro Area                             2018-08-01   
2014-06    Tampa-St. Petersburg-Clearwater, FL Metro Area      2014-06-01   
2017-03    Phoenix-Mesa-Scottsdale, AZ Metro Area              2017-03-01   
2018-05    Tampa-St. Petersburg-Clearwater, FL Metro Area      2018-05-01   
2014-03    Charlotte-Concord-Gastonia, NC-SC Metro Area        2014-03-01   
2014-12    Dallas-Fort Worth-Arlington, TX Metro Area          2014-12-01   
2017-12    Riverside-San Bernardino-Ontario, CA Metro Area     2017-12-01   
2016-09    Austin-Round Rock, TX Metro Area                    2016-09-01   
2014-09    Tampa-St. Petersburg-Clearwater, FL Metro Area      2014-09-01   
2014-02    Las Vegas-Henderson-Paradise, NV Metro Area         2014-02-01   
2016-02    Charlotte-Concord-Gastonia, NC-SC Metro Area        2016-02-01   
2015-12    Phoenix-Mesa-Scottsdale, AZ Metro Area              2015-12-01   
2015-05    Austin-Round Rock, TX Metro Area                    2015-05-01   
2018-09    Columbia, SC Metro Area                             2018-09-01   
2016-11    Tampa-St. Petersburg-Clearwater, FL Metro Area      2016-11-01   
2015-06    Austin-Round Rock, TX Metro Area                    2015-06-01   
...                                                                   ...   
2016-04    Phoenix-Mesa-Scottsdale, AZ Metro Area              2016-04-01   
2015-10    Austin-Round Rock, TX Metro Area                    2015-10-01   
2016-12    Dallas-Fort Worth-Arlington, TX Metro Area          2016-12-01   
2016-04    Tampa-St. Petersburg-Clearwater, FL Metro Area      2016-04-01   
2014-05    Tampa-St. Petersburg-Clearwater, FL Metro Area      2014-05-01   
2015-06    Phoenix-Mesa-Scottsdale, AZ Metro Area              2015-06-01   
2015-10    Orlando-Kissimmee-Sanford, FL Metro Area            2015-10-01   
2016-05    Columbia, SC Metro Area                             2016-05-01   
2015-04    Nashville-Davidson--Murfreesboro--Franklin, TN ...  2015-04-01   
2018-06    Phoenix-Mesa-Scottsdale, AZ Metro Area              2018-06-01   
2016-07    Charlotte-Concord-Gastonia, NC-SC Metro Area        2016-07-01   
2015-01    Austin-Round Rock, TX Metro Area                    2015-01-01   
2014-02    Austin-Round Rock, TX Metro Area                    2014-02-01   
2017-11    Las Vegas-Henderson-Paradise, NV Metro Area         2017-11-01   
2018-04    Phoenix-Mesa-Scottsdale, AZ Metro Area              2018-04-01   
2016-10    Dallas-Fort Worth-Arlington, TX Metro Area          2016-10-01   
2015-02    Columbia, SC Metro Area                             2015-02-01   
2016-03    Columbia, SC Metro Area                             2016-03-01   
2014-06    Columbia, SC Metro Area                             2014-06-01   
           Riverside-San Bernardino-Ontario, CA Metro Area     20

In [11]:
# Dropping irrelevant columns
combined_df = combined_df.drop(columns= ["date", "cbsa_code"])
combined_df

population  \
month_year city_description                                                 
2014-05    Riverside-San Bernardino-Ontario, CA Metro Area        4443098   
           Columbia, SC Metro Area                                 804684   
2014-01    Phoenix-Mesa-Scottsdale, AZ Metro Area                 4494803   
2018-08    Orlando-Kissimmee-Sanford, FL Metro Area               2568290   
2017-10    Las Vegas-Henderson-Paradise, NV Metro Area            2203346   
2017-08    Charlotte-Concord-Gastonia, NC-SC Metro Area           2527942   
2016-08    Las Vegas-Henderson-Paradise, NV Metro Area            2155946   
2017-06    Nashville-Davidson--Murfreesboro--Franklin, TN ...     1907486   
2017-11    Riverside-San Bernardino-Ontario, CA Metro Area        4574698   
2017-12    Dallas-Fort Worth-Arlington, TX Metro Area             7345796   
2015-07    Charlotte-Concord-Gastonia, NC-SC Metro Area           2425547   
2016-09    Riverside-San Bernardino-Ontario, CA Metro Area        4511762   
2017-05    Dallas-Fort Worth-Arlington, TX Metro Area             7345796   
2017-04    Charlotte-Concord-Gastonia, NC-SC Metro Area           2527942   
2018-08    Columbia, SC Metro Area                                 834324   
2014-06    Tampa-St. Petersburg-Clearwater, FL Metro Area         2919454   
2017-03    Phoenix-Mesa-Scottsdale, AZ Metro Area                 4753050   
2018-05    Tampa-St. Petersburg-Clearwater, FL Metro Area         3140906   
2014-03    Charlotte-Concord-Gastonia, NC-SC Metro Area           2373749   
2014-12    Dallas-Fort Worth-Arlington, TX Metro Area             6945276   
2017-12    Riverside-San Bernardino-Ontario, CA Metro Area        4574698   
2016-09    Austin-Round Rock, TX Metro Area                       2057443   
2014-09    Tampa-St. Petersburg-Clearwater, FL Metro Area         2919454   
2014-02    Las Vegas-Henderson-Paradise, NV Metro Area            2066423   
2016-02    Charlotte-Concord-Gastonia, NC-SC Metro Area           2474170   
2015-12    Phoenix-Mesa-Scottsdale, AZ Metro Area                 4585968   
2015-05    Austin-Round Rock, TX Metro Area                       2008606   
2018-09    Columbia, SC Metro Area                                 834324   
2016-11    Tampa-St. Petersburg-Clearwater, FL Metro Area         3025461   
2015-06    Austin-Round Rock, TX Metro Area                       2008606   
...                                                                   ...   
2016-04    Phoenix-Mesa-Scottsdale, AZ Metro Area                 4677382   
2015-10    Austin-Round Rock, TX Metro Area                       2008606   
2016-12    Dallas-Fort Worth-Arlington, TX Metro Area             7211508   
2016-04    Tampa-St. Petersburg-Clearwater, FL Metro Area         3025461   
2014-05    Tampa-St. Petersburg-Clearwater, FL Metro Area         2919454   
2015-06    Phoenix-Mesa-Scottsdale, AZ Metro Area                 4585968   
2015-10    Orlando-Kissimmee-Sanford, FL Metro Area               2383541   
2016-05    Columbia, SC Metro Area                                 821158   
2015-04    Nashville-Davidson--Murfreesboro--Franklin, TN ...     1828218   
2018-06    Phoenix-Mesa-Scottsdale, AZ Metro Area                 4876829   
2016-07    Charlotte-Concord-Gastonia, NC-SC Metro Area           2474170   
2015-01    Austin-Round Rock, TX Metro Area                       2008606   
2014-02    Austin-Round Rock, TX Metro Area                       1941049   
2017-11    Las Vegas-Henderson-Paradise, NV Metro Area            2203346   
2018-04    Phoenix-Mesa-Scottsdale, AZ Metro Area                 4876829   
2016-10    Dallas-Fort Worth-Arlington, TX Metro Area             7211508   
2015-02    Columbia, SC Metro Area                                 811574   
2016-03    Columbia, SC Metro Area                                 821158   
2014-06    Columbia, SC Metro Area                                 804684   
           Riverside-San Bernardino-Ontario, CA Metro Area 

In [12]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 600 entries, (2014-05, Riverside-San Bernardino-Ontario, CA Metro Area) to (2018-05, Charlotte-Concord-Gastonia, NC-SC Metro Area)
Data columns (total 3 columns):
population           600 non-null int64
actual_all_crimes    600 non-null float64
cum_returns          600 non-null float64
dtypes: float64(2), int64(1)
memory usage: 15.9+ KB


# Multiple Linear Regression

In [13]:
correlations = combined_df.corr(method='pearson')
correlations

,population,actual_all_crimes,cum_returns
population,1.000000,0.419031,0.181291
actual_all_crimes,0.419031,1.000000,0.095901
cum_returns,0.181291,0.095901,1.000000


In [14]:
predictors = ['population', 'actual_all_crimes']
outcome = 'cum_returns'

x = pd.get_dummies(combined_df[predictors], drop_first=True)
y = combined_df[outcome]
train_x, valid_x, train_y, valid_y = train_test_split(x,y, test_size = 0.1, random_state=1)

housing_lm = LinearRegression()
housing_lm.fit(train_x, train_y)

LinearRegression()

In [15]:
print(pd.DataFrame({'Predictor': x.columns, 'coffecients': housing_lm.coef_}))

           Predictor   coffecients
0         population  2.811121e-10
1  actual_all_crimes  7.216945e-09


In [16]:
train_df = train_x.join(train_y)

formula = 'cum_returns ~ ' + '+'. join(predictors)
print(formula)

housing_lm = smf.ols(formula=formula, data=train_df).fit()
housing_lm.summary()

cum_returns ~ population+actual_all_crimes


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            cum_returns   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     9.695
Date:                Mon, 11 Jan 2021   Prob (F-statistic):           7.30e-05
Time:                        12:36:28   Log-Likelihood:                 2398.0
No. Observations:                 540   AIC:                            -4790.
Df Residuals:                     537   BIC:                            -4777.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.0049      0.000     19.429      0.000       0.004       0.005
population         2.811e-10   7.67e-11      3.664      0.000     1.3e-10    4.32e-10
actual_all_crimes  7.217e-09   1.13e-08      0.637      0.524    -1.5e-08    2.95e-08
==============================================================================
Omnibus:                       57.061   Durbin-Watson:                   2.059
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              152.890
Skew:                           0.525   Prob(JB):                     6.32e-34
Kurtosis:                       5.386   Cond. No.                     7.43e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.43e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""